In [160]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Data Loading and Preprocessing

In [161]:
compas_df=pd.read_csv("C:\\Users\\tqy52\\Desktop\\Courses\\Applied Data Science\\Project4\\compas-scores-two-years.csv")

#Use the data of African American and Caucasian

races = ['African-American','Caucasian']

compas_df=compas_df[compas_df.race.isin(races)]

compas_df.head()

#use only specific columns for future prediction
compas_df_filtered = compas_df.loc[:, ['race']].join(compas_df.select_dtypes(include=['int64']))
#drop id and duplicate columns
compas_df_filtered = compas_df_filtered.drop(['id','decile_score.1', 'priors_count.1'], axis=1)

# split the data into train, validation and test set, ratio 5:1:1

# Split data into training and validation/test sets
train_df, val_test_df = train_test_split(compas_df_filtered, test_size=0.2, random_state=42)

# Split validation/test set into validation and test sets
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42)


In [162]:

print(train_df.dtypes)


race                object
age                  int64
juv_fel_count        int64
decile_score         int64
juv_misd_count       int64
juv_other_count      int64
priors_count         int64
is_recid             int64
is_violent_recid     int64
v_decile_score       int64
start                int64
end                  int64
event                int64
two_year_recid       int64
dtype: object


# Implemntaion of algorithms

In [163]:
#for the algorithms below, we apply on the whole dataset
X=compas_df_filtered
s=X.race
e=X.decile_score 
y=X.two_year_recid 




def PARTITION(X, e):
    unique_values = e.unique()  # find unique values of e
    groups = {}  # initialize empty dictionary to store resulting DataFrames
    
    for val in unique_values:
        group_df = X[e == val]  # filter rows in X where e equals current unique value
        groups[val] = group_df  # store resulting DataFrame in dictionary with key = unique value
        
    return groups  # return dictionary of resulting DataFrames







In [164]:
resulting_groups = PARTITION(X, e)
first_group_df = resulting_groups[1]
#print(first_group_df.head)

unique_values = e.unique() 
print(unique_values)

[ 3  4  8  6  1 10  5  9  2  7]


In [167]:

def delta(X, gender):
    # calculate the baseline probabilities
    p_pos_base = X.two_year_recid.mean()

    # calculate probabilities conditioned on gender
    p_pos_gender = X[X.race == gender].two_year_recid.mean()
    

    # calculate the number of gender people in X
    G = len(X[X.race == gender])

    # calculate delta
    
    delta_val = G * abs(p_pos_gender - p_pos_base)
    

    return int(delta_val)


In [168]:
for group in resulting_groups.values():
    print(delta(group,'Caucasian'))

11
10
3
1
5
4
3
1
1
2


In [189]:

#Algorithm 1: Local massaging
resulting_groups = PARTITION(X, e)
updated_groups_1 = []
for group in resulting_groups.values():
    males = group[group.race == 'Caucasian']
    females = group[group.race == 'African-American']
    
    # learn a ranker H for this group
    H = LogisticRegression(random_state=42,max_iter=500)
    H.fit(group.drop(['race','two_year_recid'], axis=1), group.two_year_recid)
    
    # rank and relabel males
    males_females = pd.concat([males,females])
    males_females['proba'] = H.predict_proba(males_females.drop(['two_year_recid','race'], axis=1))[:, 1]

   
    males_females = males_females.sort_values(by='proba')
    
    
    delta_males = int(delta(group, 'Caucasian'))
 
    males_to_relabel = males_females[(males_females['race'] == 'Caucasian') & (males_females['two_year_recid'] == 0) & (males_females['proba'] < 0.5)].head(delta_males)
    group.loc[males_to_relabel.index, 'two_year_recid'] = 1 - males_to_relabel.two_year_recid
    
    # rank and relabel females
    delta_females = int(delta(group, 'African-American'))
    females_to_relabel = males_females[(males_females['race'] == 'African-American') & (males_females['two_year_recid'] == 1) & (males_females['proba'] > 0.5)].head(delta_females)
    group.loc[females_to_relabel.index, 'two_year_recid'] = 1 - females_to_relabel.two_year_recid
    
    updated_groups_1.append(group)


c:\users\tqy52\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
c:\users\tqy52\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
c:\users\tqy52\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:1676: Setting

In [190]:
updated_df_1 = pd.concat(updated_groups_1)
updated_df_1 = updated_df_1.sort_index()
print(sum(updated_df_1['two_year_recid']!=compas_df_filtered['two_year_recid']))
#67 labels have been changed

67


0


In [191]:

#Algorithm 2:  Local preferential sampling
resulting_groups = PARTITION(X, e)
updated_groups_2 = []
for group in resulting_groups.values():
    males = group[group.race == 'Caucasian']
    females = group[group.race == 'African-American']
    
    # learn a ranker H for this group
    H = LogisticRegression(random_state=42,max_iter=500)
    H.fit(group.drop(['race','two_year_recid'], axis=1), group.two_year_recid)
    
    # rank delete,and relabel males
    males_females = pd.concat([males, females])
    males_females['proba'] = H.predict_proba(males_females.drop(['two_year_recid', 'race'], axis=1))[:, 1]
    males_females = males_females.sort_values(by='proba')
    delta_males = int(0.5*delta(group, 'Caucasian'))
    
    
    males_to_delete = males_females[(males_females['race'] == 'Caucasian') & (males_females['two_year_recid'] == 1) & (males_females['proba'] < 0.5)].tail(delta_males)
    males_to_duplicate = males_females[(males_females['race'] == 'Caucasian') & (males_females['two_year_recid'] == 0) & (males_females['proba'] > 0.5)].head(delta_males)
    group = group[~group.index.isin(males_to_delete.index)]
    group = pd.concat([group, males_to_duplicate], ignore_index=True)
    
    # rank delete,and relabel females
    delta_females = int(0.5*delta(group, 'African-American'))
    females_to_delete = males_females[(males_females['race'] == 'African-American') & (males_females['two_year_recid'] == 0) & (males_females['proba'] > 0.5)].tail(delta_females)
    females_to_duplicate = males_females[(males_females['race'] == 'African-American') & (males_females['two_year_recid'] == 1) & (males_females['proba'] < 0.5)].head(delta_females)
    group = group[~group.index.isin(females_to_delete.index)]
    group = pd.concat([group, females_to_duplicate], ignore_index=True)
    
    updated_groups_2.append(group)
    
   


In [197]:
updated_df_2 = pd.concat(updated_groups_2)






0       1
0       1
0       0
0       1
0       0
       ..
1075    1
1076    0
1077    0
1078    0
1079    1
Name: two_year_recid, Length: 6159, dtype: int64
